In [63]:
from difflib import get_close_matches
import pandas as pd
from mobfot import MobFot
from difflib import get_close_matches
client  = MobFot()

def parse_time(time):
    return f"{time.replace('T', ' ').rsplit('.', 1)[0]}"


ids = list()
for x in client.get_league(77)['matches']['allMatches']:
    ids.append(x['id'])


df = pd.read_csv("FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv")
df['match_id'] = None

home_teams = df.home_team.unique()
away_teams = df.away_team.unique()
for id in ids:
    match = client.get_match_details(id)['general']
    parsed_time = parse_time(match['matchTimeUTCDate'])
    date_string = parsed_time.split(' ')[0]
    time_string = parsed_time.split(' ')[1]
    time_string = str((int(time_string.split(':')[0]) + 3) % 24) + ':00:00'
    time = date_string + ' ' + time_string
    home_team = match['homeTeam']['name']
    if home_team == 'USA':
        home_team = 'United States'
    if home_team == 'South Korea':
        home_team = 'Korea Republic'
    home_team = get_close_matches(home_team, home_teams, n=1, cutoff=0.5)[0]
    
    away_team = match['awayTeam']['name']
    if away_team == 'USA':
        away_team = 'United States'
    if away_team == 'South Korea':
        away_team = 'Korea Republic'
    away_team = get_close_matches(away_team, away_teams, n=1, cutoff=0.5)[0]
    print(f"Currently adding {time} {home_team} vs {away_team}")
    df.loc[(df.match_time == time) & (df.home_team == home_team) & (df.away_team == away_team), 'match_id'] = id


df.to_csv("FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv", index=False)

def add_subs(match, homePlayers, awayPlayers):
    for index_team, team in enumerate(match['content']['lineup']['bench']['benchArr']):
        for player in team:
            if player['minutesPlayed'] > 0:
                if index_team == 0:
                    homePlayers.append(player)
                else:
                    awayPlayers.append(player)
                
    


def get_player_data(match):
    homePlayers = list()
    for x in match['content']['lineup']['lineup'][0]['players']:
        for y in x:
            homePlayers.append(y)

    awayPlayers = list()
    for x in match['content']['lineup']['lineup'][1]['players']:
        for y in x:
            awayPlayers.append(y)
    
    add_subs(match, homePlayers, awayPlayers)  
    return homePlayers, awayPlayers



        

id = 3854559
match = client.get_match_details(id)

homePlayers, awayPlayers = get_player_data(match)

def parse_api_results(player):
    if not player['isGoalkeeper']:
        name = player['name']['fullName']
        if 'positionStringShort' not in player.keys():
            position = 'substitute'
        else:
            position = player['positionStringShort']
        rating  = player['rating']['num']
        shotmap = player['shotmap']
        role = player['role'].lower()
        stats = player['stats']
        shotmaps = []
        for y in shotmap:
            current_shotmap = {}
            current_shotmap['eventType'] = y['eventType']
            current_shotmap['isOnTarget'] = y['isOnTarget']
            current_shotmap['min'] = y['min']
            current_shotmap['situation'] = y['situation']
            current_shotmap['x'] = y['x']
            current_shotmap['y'] = y['y']
            current_shotmap['goalCrossedY'] = y['goalCrossedY']
            current_shotmap['goalCrossedZ'] = y['goalCrossedZ']
            shotmaps.append(current_shotmap)
        
        
        
        for x in stats:
            if x['key'] == 'top_stats':
                top_stats = x
                new_top_stats = dict()
                for y in top_stats['stats']:
                    new_top_stats[top_stats['stats'][y]['key']] = top_stats['stats'][y]['value']
                top_stats = new_top_stats
            elif x['key'] == 'attack':
                attack_stats = x
                new_attack_stats = dict()
                for y in attack_stats['stats']:
                    new_attack_stats[attack_stats['stats'][y]['key']] = attack_stats['stats'][y]['value']
                attack_stats = new_attack_stats
            elif x['key'] == 'defense':
                defense_stats = x
                new_defense_stats = dict()
                for y in defense_stats['stats']:
                    new_defense_stats[defense_stats['stats'][y]['key']] = defense_stats['stats'][y]['value']
                defense_stats = new_defense_stats
            elif x['key'] == 'duels':
                duel_stats = x
                new_duel_stats = dict()
                for y in duel_stats['stats']:
                    new_duel_stats[duel_stats['stats'][y]['key']] = duel_stats['stats'][y]['value']
                duel_stats = new_duel_stats
        
        new_results = {
        'name': name,
        'position': position,
        'rating': rating,
        'role': role,
        'top_stats': top_stats,
        'attack_stats': attack_stats,
        'defense_stats': defense_stats,
        'duel_stats': duel_stats,
        'shotmaps': shotmaps
    }
    else:
        name = player['name']['fullName']
        if 'positionStringShort' not in player.keys():
            position = 'substitute'
        else:
            position = player['positionStringShort']
        
        rating  = player['rating']['num']
        role = player['role'].lower()
        top_stats = player['stats'][0]  
        new_results = {
        'name': name,
        'position': position,
        'rating': rating,
        'role': role,
        'top_stats': top_stats
    }
    
    
    return new_results
    
    
homePlayers = [parse_api_results(x) for x in homePlayers]
awayPlayers = [parse_api_results(x) for x in awayPlayers]





Currently adding 2022-11-20 19:00:00 Qatar vs Ecuador
Currently adding 2022-11-21 16:00:00 England vs IR Iran
Currently adding 2022-11-21 19:00:00 Senegal vs Netherlands
Currently adding 2022-11-21 22:00:00 United States vs Wales
Currently adding 2022-11-22 13:00:00 Argentina vs Saudi Arabia
Currently adding 2022-11-22 16:00:00 Denmark vs Tunisia
Currently adding 2022-11-22 19:00:00 Mexico vs Poland
Currently adding 2022-11-22 22:00:00 France vs Australia
Currently adding 2022-11-23 13:00:00 Morocco vs Croatia
Currently adding 2022-11-23 16:00:00 Germany vs Japan
Currently adding 2022-11-23 19:00:00 Spain vs Costa Rica
Currently adding 2022-11-23 22:00:00 Belgium vs Canada
Currently adding 2022-11-24 13:00:00 Switzerland vs Cameroon
Currently adding 2022-11-24 16:00:00 Uruguay vs Korea Republic
Currently adding 2022-11-24 19:00:00 Portugal vs Ghana
Currently adding 2022-11-24 22:00:00 Brazil vs Serbia
Currently adding 2022-11-25 13:00:00 Wales vs IR Iran
Currently adding 2022-11-25 16:

In [66]:
len(awayPlayers)

17

In [2]:

import datetime
from mobfot import MobFot
def get_team_country(id):
    client = MobFot()
    try:
        return client.get_team(id)['details']['country']
        
    except:
        return None

def parse_date(date):
    # create time object of string with format 2013-07-01T00:00:00
    time = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')
    return time


def get_player_club_history(id):
    client = MobFot()
    clubHistory = {}
    total_hisorty = []
    for y in client.get_player(id)['careerHistory']['careerItems']['senior']['teamEntries']:
        clubHistory['name'] = client.get_player(x)['name']
        clubHistory['teamId'] = y['teamId']
        clubHistory['teamName'] = y['team']
        clubHistory['country'] = get_team_country(y['teamId'])
        clubHistory['startDate'] = parse_date(y['startDate'])
        if not y['endDate']:
            clubHistory['endDate'] = datetime.datetime.now()
        else:
            clubHistory['endDate'] = parse_date(y['endDate'])
        total_hisorty.append(clubHistory)
    return total_hisorty




get_player_club_history('466852')

NameError: name 'x' is not defined

In [15]:

# get match momentum
from mobfot import MobFot

client = MobFot()
wedstrijd = client.get_match_details(3854556)

wedstrijd['content']['momentum']


def get_momentum(momentum):
    momentum_list = []
    momentum_minutes = []
    momentum = momentum['main']['data']
    for x in momentum:
        momentum_list.append(x['value'])
        momentum_minutes.append(x['minute'])
        
    return momentum_list, momentum_minutes
        
    
mom_list, mom_mins = get_momentum(wedstrijd['content']['momentum'])


print(*list(zip(mom_mins, mom_list)), sep='\n')

(0, 0)
(1, -5)
(2, 5)
(3, 5)
(4, 11)
(5, 5)
(6, 5)
(7, 5)
(8, 0)
(9, -11)
(10, -16)
(11, -33)
(12, -16)
(13, -11)
(14, 11)
(15, 5)
(16, 5)
(17, 11)
(18, 22)
(19, 33)
(20, 11)
(21, 0)
(22, -16)
(23, 0)
(24, -16)
(25, -49)
(26, -54)
(27, -100)
(28, -100)
(29, -100)
(30, 5)
(31, 5)
(32, 22)
(33, 11)
(34, 11)
(35, 0)
(36, 0)
(37, 5)
(38, 11)
(39, 5)
(40, 16)
(41, 11)
(42, 0)
(43, -16)
(44, -11)
(45, 5)
(45.5, -35)
(46, 0)
(47, 0)
(48, -5)
(49, -11)
(50, -11)
(51, 0)
(52, 22)
(53, 22)
(54, 22)
(55, 5)
(56, -5)
(57, -16)
(58, -16)
(59, -54)
(60, -43)
(61, -43)
(62, -11)
(63, 5)
(64, -33)
(65, -16)
(66, -27)
(67, 16)
(68, 16)
(69, 5)
(70, 5)
(71, 22)
(72, 22)
(73, 22)
(74, 5)
(75, 11)
(76, 16)
(77, 5)
(78, 0)
(79, -11)
(80, -11)
(81, -16)
(82, -16)
(83, 5)
(84, 5)
(85, 5)
(86, -5)
(87, 22)
(88, 22)
(89, 22)
(90, 0)
(90.5, -12)
(90.75, 0)


In [60]:
from icecream import ic
match = client.get_match_details(id)
for team in match['content']['lineup']['bench']['benchArr']:
    for player in team:
        if player['minutesPlayed'] > 0:
            
            ic(player)
            break

ic| player: {'events': {'g': 1, 'sub': {'subbedIn': 70}},
             'fantasyScore': {'bgcolor': 'purple', 'num': None},
             'id': '312765',
             'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/312765.png',
             'isCaptain': False,
             'isGoalkeeper': False,
             'isHomeTeam': True,
             'localizedPosition': None,
             'minutesPlayed': 19,
             'name': {'firstName': 'Jack',
                      'fullName': 'Jack Grealish',
                      'lastName': 'Grealish'},
             'pageUrl': '/players/312765/jack-grealish',
             'position': 'Substitute',
             'positionId': 0,
             'positionRow': 0,
             'rating': {'bgcolor': '#1ec853',
                        'isTop': {'isMatchFinished': True, 'isTopRating': False},
                        'num': '7.7'},
             'role': 'Attacker',
             'shirt': 7,
             'shotmap': [{'blockedX': None,
           